In [2]:
!pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/bc/0d/486e3fa27f39a00168abfcf14a3d8444f437f4b755cc34316da1124f293d/psycopg2_binary-2.9.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import psycopg2 as pg

In [4]:
    - *dbname*: the database name
    - *database*: the database name (only as keyword argument)
    - *user*: user name used to authenticate
    - *password*: password used to authenticate
    - *host*: database host address (defaults to UNIX socket if not provided)
    - *port*: connection port number (defaults to 5432 if not provided)

SyntaxError: invalid syntax (2917335226.py, line 1)

In [5]:
jdbc:postgresql://database-2.cluster-cy1raopcpjus.us-east-1.rds.amazonaws.com:5432/db1

SyntaxError: invalid decimal literal (280278825.py, line 1)

In [45]:
import getpass
user = 'postgres'
dbname = 'db1'
host = 'database-1.c0oohyxv9j5t.us-east-1.rds.amazonaws.com'
port = '5432'
#password=getpass.getpass()
password = 'Password123'

In [46]:
conn = pg.connect(dbname=dbname,host=host, port=port,user=user,password=password)
cursor = conn.cursor()

In [47]:
 event_table_description = (
        "CREATE TABLE sport_event ("
        "  event_id SERIAL PRIMARY KEY,"
        "  sport_type VARCHAR(256) NOT NULL,"
        "  start_date timestamp NOT NULL,"
        "  location VARCHAR(256) NOT NULL"
       
        ")"
    )

In [48]:
#print("Creating table {}: ".format("sport_event"), end='')
cursor.execute(event_table_description)

In [49]:
conn.commit()

In [50]:
 ticket_table_description = (
        "CREATE TABLE ticket ("
        "  ticket_id SERIAL PRIMARY KEY,"
        "  event_id INT NOT NULL,"
        "  seat_level VARCHAR(256) NOT NULL,"
        "  seat_location VARCHAR(256) NOT NULL,"
        "  ticket_price INT NOT NULL,"

        "  FOREIGN KEY (event_id) REFERENCES sport_event(event_id)"
        ")"
    )

In [51]:
cursor.execute(ticket_table_description)

In [52]:
conn.commit()

In [53]:
sql =""" SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';"""

In [54]:
cursor.execute(sql)

In [55]:
cursor.fetchall()

[('public', 'sport_event', 'postgres', None, True, False, True, False),
 ('public', 'ticket', 'postgres', None, True, False, True, False)]

In [56]:
conn.commit()

In [57]:
customer_table_description = (
        "CREATE TABLE customer ("
        "  customer_id SERIAL PRIMARY KEY,"
        "  customer_name VARCHAR(256) NOT NULL,"
        "  email_address VARCHAR(256) NOT NULL,"
        "  phone_number VARCHAR(256) NOT NULL"
      
        ")"
    )

In [58]:
cursor.execute(customer_table_description)
conn.commit()

In [59]:
 ticket_activity_table_description = (
        "CREATE TABLE ticket_activity ("
        "  ticket_id INT NOT NULL,"
        "  purchased_by INT NOT NULL,"
        "  created_at timestamp NOT NULL,"
        "  updated_at timestamp NOT NULL,"
        "  PRIMARY KEY (ticket_id),"
        "  FOREIGN KEY (ticket_id) REFERENCES ticket(ticket_id),"
        "  FOREIGN KEY (purchased_by) REFERENCES customer(customer_id)"
        ")"
    )
    
cursor.execute(ticket_activity_table_description)
conn.commit()

In [60]:
sport_type_list = [
    "Baseball",
    "Football"
]

location_list = [
    "Seattle, US",
    "New York, US",
    "San Francisco, US",
    "Los Angeles, US",
    "Boston, US",
    "Chicago, US"
]

In [61]:
def generate_seat_list(num):
    seat_level_price = {
        "Standard": 100,
        "Premium": 300
    }
    list = []
    for seat_level in seat_level_price.keys():
        for i in range(num):
            list.append((seat_level, f"{seat_level[0]}-{i}", seat_level_price[seat_level]))
    return list

In [31]:
!pip install faker

  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/9b/69/fc3c9c30d406655d00f9b5737f0a27f7ba89d8fff61c14d5ecd223acf211/Faker-19.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [62]:
import sys
import random
from faker import Faker
from datetime import datetime, timedelta
#import mysql.connector
#from mysql.connector import errorcode
#from awsglue.utils import getResolvedOptions

In [63]:


def initialize_data(cursor, connection):
    today = datetime.today()

    try:
        print("Initializing data in table `sport_event` and `ticket`:", end='')
        add_event = ("INSERT INTO sport_event "
                     "(sport_type, start_date, location) "
                     "VALUES (%s, %s, %s)")
        for num in range(50):
            future_date = (today + timedelta(days=random.randint(30,365))).date()
            data_event = (random.choice(sport_type_list), future_date, random.choice(location_list))
            cursor.execute(add_event, data_event)
            latest_event_id = cursor.lastrowid+1

            add_ticket = ("INSERT INTO ticket "
                          "(event_id, seat_level, seat_location, ticket_price) "
                          "VALUES (%s, %s, %s, %s)")
            for seat_level, seat_location, ticket_price in generate_seat_list(100):
                data_ticket = (latest_event_id, seat_level, seat_location, ticket_price)
                cursor.execute(add_ticket, data_ticket)
    except Exception as err:
        print(err.msg)
    else:
        print("OK")

    try:
        print("Ingesting data in table `customer`:", end='')
        add_customer = ("INSERT INTO customer "
                        "(customer_name, email_address, phone_number) "
                        "VALUES (%s, %s, %s)")

        for num in range(1000):
            data_customer = (fake.name(), fake.ascii_email(), fake.phone_number())
            cursor.execute(add_customer, data_customer)
    except Exception as err:
        print(err.msg)
    else:
        print("OK")

    connection.commit()



In [64]:

def ingest_data(cursor, connection):
    today = datetime.today()

    print("Selecting one record from table {}".format("ticket"))
    cursor.execute("SELECT ticket.ticket_id FROM ticket LEFT OUTER JOIN ticket_activity on ticket.ticket_id = ticket_activity.ticket_id WHERE ticket_activity.ticket_id is NULL ORDER BY RANDOM() LIMIT 1")
    rows = cursor.fetchall()
    ticket_id = 0
    for row in rows:
        ticket_id = row[0]
    print(f"ticket id = {ticket_id}")
    print("Selecting one record from table {}".format("customer"))
    cursor.execute("SELECT customer_id FROM customer ORDER BY RANDOM() LIMIT 1")
    rows = cursor.fetchall()
    customer_id = 0
    for row in rows:
        customer_id = row[0]
    print(f"customer id = {customer_id}")
    print("Ingesting data into table {}: ".format("ticket_activity"), end='')
    add_ticket_activity = ("INSERT INTO ticket_activity "
                 "(ticket_id, purchased_by, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s)")

    data_ticket_activity = (ticket_id, customer_id, today, today)
    print(data_ticket_activity)
    cursor.execute(add_ticket_activity, data_ticket_activity)

    connection.commit()





In [35]:
fake = Faker()

In [65]:
initialize_data(cursor,conn)

Initializing data in table `sport_event` and `ticket`:OK
Ingesting data in table `customer`:OK


In [66]:
i=0
while i < 2:
    ingest_data(cursor,conn)
    i=i+1

Selecting one record from table ticket
ticket id = 3633
Selecting one record from table customer
customer id = 602
Ingesting data into table ticket_activity: (3633, 602, datetime.datetime(2023, 10, 16, 19, 3, 40, 582861), datetime.datetime(2023, 10, 16, 19, 3, 40, 582861))
Selecting one record from table ticket
ticket id = 7976
Selecting one record from table customer
customer id = 528
Ingesting data into table ticket_activity: (7976, 528, datetime.datetime(2023, 10, 16, 19, 3, 40, 590606), datetime.datetime(2023, 10, 16, 19, 3, 40, 590606))


In [68]:
conn.close()

In [76]:
conn = pg.connect(dbname=dbname,host=host, port=port,user=user,password=password)
cursor = conn.cursor()

In [70]:
sql="""CREATE TABLE demo_json (
	cust_id int NOT NULL,
	info json NOT NULL,
    created_at timestamp NOT NULL,
          updated_at timestamp NOT NULL,
         primary key (cust_id)
    
);"""

cursor.execute(sql)
conn.commit()

In [71]:
today = datetime.today()
add_customer = ("INSERT INTO demo_json "
                        "(cust_id,info,created_at,updated_at) "
                        "VALUES (%s,%s, %s,%s)")
values='{ "customer": "John Doe", "items": {"product": "Beer","qty": 6}}'
data_customer = (10,values,today,today)
cursor.execute(add_customer, data_customer)
conn.commit()

In [73]:
sql="""CREATE TABLE demo_1 (
          cust_id int not null,
          first_name VARCHAR(256) NOT NULL,
         last_name VARCHAR(256) NOT NULL,
         created_at timestamp NOT NULL,
          updated_at timestamp NOT NULL,
         primary key (cust_id)
        
      
        )"""

cursor.execute(sql)
conn.commit()

DuplicateTable: relation "demo_1" already exists


In [77]:
today = datetime.today()
add_customer = ("update demo_1 set last_name = %s, updated_at = %s where cust_id = 10")

data_customer = ("orange",today)
cursor.execute(add_customer, data_customer)
conn.commit()

In [78]:
today = datetime.today()
add_customer = ("INSERT INTO demo_1 "
                        "(cust_id,first_name, last_name,created_at,updated_at) "
                        "VALUES (%s,%s, %s,%s,%s)")

data_customer = (10,"purple","pink",today,today)
cursor.execute(add_customer, data_customer)
conn.commit()

In [79]:
sql="""CREATE TABLE demo (
          id SERIAL PRIMARY KEY,
          firstr_name VARCHAR(256) NOT NULL,
         last_name VARCHAR(256) NOT NULL
        
      
        )"""

cursor.execute(sql)
conn.commit()

In [80]:
add_customer = ("INSERT INTO demo "
                        "(firstr_name, last_name) "
                        "VALUES (%s, %s)")

data_customer = ("john","doe")
cursor.execute(add_customer, data_customer)
conn.commit()

In [81]:
cursor.execute("truncate table ticket_activity")

In [23]:
cursor.execute("drop table demo_1")

In [117]:
cursor.execute("drop table ticket")

In [119]:
cursor.execute("drop table sport_event")

In [24]:
conn.commit()

In [14]:
 today = datetime.today()

In [ ]:
!aws s3 ls s3://aws-bigdata-blog/artifacts/

In [168]:
cursor.execute("""SELECT sequence_schema, sequence_name 
FROM information_schema.sequences 
ORDER BY sequence_name """)

In [13]:
cursor.execute("""select count(*) from ticket_activity""")
cursor.fetchall()

[(8,)]

In [189]:
cursor.execute("""ALTER SEQUENCE customer_customer_id_seq
RESTART WITH 1;""")

In [190]:
cursor.execute("""ALTER SEQUENCE sport_event_event_id_seq
RESTART WITH 1;""")

In [191]:
cursor.execute("""ALTER SEQUENCE ticket_ticket_id_seq
RESTART WITH 1;""")

In [192]:
conn.commit()

In [ ]:
SELECT id, name, employee.deptno, deptname
    FROM employee INNER JOIN department ON employee.deptno = department.deptno;

In [ ]:
select ticketactivity.ticket_id,purchased_by, created_at , updated_at ,event_id,seat_level,seat_location,ticket_price
               from ticketactivity inner join ticket on ticketactivity.ticket_id=ticket.ticket_id

In [ ]:
select *  from ta inner join cu on ta.purchaed_by=cu.customer_id

In [38]:
!aws glue reset-job-bookmark --job-name demo_2

{
    "JobBookmarkEntry": {
        "JobName": "demo_2",
        "Version": 7,
        "Run": 5,
        "Attempt": 0,
        "JobBookmark": ""
    }
}
